# Case 1: No reserve requirements

In [4]:
using CSV, DataFrames, JuMP, HiGHS, SCIP, Clp

storageinfo = CSV.read(joinpath("cleaned","storageinfo.csv"), DataFrame)
generatorinfo = CSV.read(joinpath("cleaned","generatorinfo.csv"), DataFrame)
vreinfo = CSV.read(joinpath("cleaned","vreinfo.csv"), DataFrame)
halfhourlydemand = CSV.read(joinpath("cleaned","halfhourlydemand.csv"), DataFrame)
halfhourlyvrecf = CSV.read(joinpath("cleaned","halfhourlyvrecf.csv"), DataFrame)
halfhourlydemand = halfhourlydemand[1:1440,:];

## Scheme 1 (No storage)

In [5]:
function case1scheme1(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
    budget, voll, bigm, epsilon)

    # SETS
    V = vreinfo.id
    G = generatorinfo.id
    S = storageinfo.id
    T = halfhourlydemand.hh
    T1 = T[2:end]

    # INITIATE MODEL
    model = Model()
    set_optimizer(model, HiGHS.Optimizer)

    # DECISION VARIABLES
    @variables(model, begin
        CAPG[G] >= 0
        GEN[G,T] >= 0
        CURTAIL[V,T] >= 0
        NSE[T] >= 0
    end)

    # BUDGET CONSTRAINT
    @expression(model, GenInvCost, sum(generatorinfo[generatorinfo.id.==g,:fixedCost][1] * CAPG[g] for g in G))
    @expression(model, TIC, GenInvCost)
    @constraint(model, cBudget, TIC <= budget)

    # OBJECTIVE FUNCTION
    @expression(model, EnergyProvisionCost, 
        sum(0.5 * generatorinfo[generatorinfo.id.==g,:varCost][1] * GEN[g,t] for g in G for t in T))
    @expression(model, UnservedEnergyCost, 
        sum(0.5 * voll * NSE[t] for t in T))
    @expression(model, CurtailedVreCost,
        sum(0.5 * vreinfo[vreinfo.id.==v,:varCost][1] * CURTAIL[v,t] for v in V for t in T))
    @objective(model, Min, 
        EnergyProvisionCost + UnservedEnergyCost + CurtailedVreCost)

    # POWER BALANCE CONSTRAINT
    @constraint(model, c01[t in T], sum(GEN[g,t] for g in G) + NSE[t] - sum(CURTAIL[v,t] for v in V) +
        sum(vreinfo[vreinfo.id.==v,:installedMW][1] * 
        halfhourlyvrecf[(halfhourlyvrecf.hh.==t) .& (halfhourlyvrecf.vre_id.==v),:cf][1] for v in V) == 
        halfhourlydemand[halfhourlydemand.hh.==t,:load][1])
    
    # GENERATOR LIMIT CONSTRAINT
    @constraint(model, c90[g in G, t in T], GEN[g,t] <= CAPG[g])

    # GENERATOR RAMP UP CONSTRAINT
    @constraint(model, c11[g in G, t in T1], 
        GEN[g,t] - GEN[g,t-1] <= generatorinfo[generatorinfo.id.==g,:rampupRate][1] * CAPG[g])
    
    # GENERATOR RAMP DOWN CONSTRAINT
    @constraint(model, c12[g in G, t in T1],
        GEN[g,t-1] - GEN[g,t] <= generatorinfo[generatorinfo.id.==g,:rampdownRate][1] * CAPG[g])
    
    # VRE CURTAIL CONSTRAINT
    @constraint(model, c13[v in V, t in T], CURTAIL[v,t] <= 
        vreinfo[vreinfo.id.==v,:installedMW][1] * 
        halfhourlyvrecf[(halfhourlyvrecf.hh.==t) .& (halfhourlyvrecf.vre_id.==v),:cf][1])

    optimize!(model)

    return(
        CAPG = value.(CAPG).data,
        GEN = value.(GEN).data,
        CURTAIL = value.(CURTAIL).data,
        NSE = value.(NSE).data,
        GenInvCost = value(GenInvCost),
        TIC = value(TIC),
        EnergyProvisionCost = value(EnergyProvisionCost),
        UnservedEnergyCost = value(UnservedEnergyCost),
        CurtailedVreCost = value(CurtailedVreCost),
        OV = objective_value(model),
    )
end

case1scheme1 (generic function with 1 method)

In [3]:
c1s1b10 = case1scheme1(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
budget=10e9, voll=1e6, bigm=100e3, epsilon=0.05)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
10077 rows, 7254 cols, 30282 nonzeros
10076 rows, 7141 cols, 30168 nonzeros
Presolve : Reductions: rows 10076(-4321); columns 7141(-1501); elements 30168(-5822)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     2.5399500000e+05 Pr: 1440(2.36784e+07) 0s
       2285     1.3001426396e+09 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 2285
Objective value     :  1.3001426396e+09
HiGHS run time      :          0.09


(CAPG = [47979.29032258065, 1840.1096774193518], GEN = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], CURTAIL = [-0.0 -0.0 … -0.0 -0.0; 0.0 0.0 … 0.0 0.0; 17712.800000000003 17067.800000000003 … 202.59999999999854 1117.5999999999985], NSE = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], GenInvCost = 1.0e10, TIC = 1.0e10, EnergyProvisionCost = 1.2803715362323117e9, UnservedEnergyCost = 0.0, CurtailedVreCost = 1.9771103400000032e7, OV = 1.300142639632312e9)

In [7]:
c1s1b20 = case1scheme1(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
budget=20e9, voll=1e6, bigm=100e3, epsilon=0.05)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
10077 rows, 7254 cols, 30282 nonzeros
10076 rows, 7141 cols, 30168 nonzeros
Presolve : Reductions: rows 10076(-4321); columns 7141(-1501); elements 30168(-5822)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     2.5399500000e+05 Pr: 1440(2.36784e+07) 0s
       1441     9.3650015940e+08 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 1441
Objective value     :  9.3650015940e+08
HiGHS run time      :          0.04


(CAPG = [13144.666666666666, 49819.4], GEN = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], CURTAIL = [-0.0 -0.0 … -0.0 -0.0; 0.0 0.0 … 0.0 0.0; 17712.800000000003 17067.800000000003 … 202.59999999999854 1117.5999999999985], NSE = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], GenInvCost = 2.0e10, TIC = 2.0e10, EnergyProvisionCost = 9.16729056e8, UnservedEnergyCost = 0.0, CurtailedVreCost = 1.9771103400000032e7, OV = 9.365001594000006e8)

In [6]:
c1s1b30 = case1scheme1(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
budget=30e9, voll=1e6, bigm=100e3, epsilon=0.05)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
10077 rows, 7254 cols, 30282 nonzeros
10076 rows, 7141 cols, 30168 nonzeros
Presolve : Reductions: rows 10076(-4321); columns 7141(-1501); elements 30168(-5822)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     2.5399500000e+05 Pr: 1440(2.36784e+07) 0s
       1441     9.3650015940e+08 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 1441
Objective value     :  9.3650015940e+08
HiGHS run time      :          0.03


(CAPG = [64426.717948717946, 49819.4], GEN = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], CURTAIL = [-0.0 -0.0 … -0.0 -0.0; 0.0 0.0 … 0.0 0.0; 17712.800000000003 17067.800000000003 … 202.59999999999854 1117.5999999999985], NSE = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], GenInvCost = 3.0e10, TIC = 3.0e10, EnergyProvisionCost = 9.16729056e8, UnservedEnergyCost = 0.0, CurtailedVreCost = 1.9771103400000032e7, OV = 9.365001594000006e8)

In [6]:
c1s1b15 = case1scheme1(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
budget=15e9, voll=1e6, bigm=100e3, epsilon=0.05)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
10077 rows, 7254 cols, 30282 nonzeros
10076 rows, 7141 cols, 30168 nonzeros
Presolve : Reductions: rows 10076(-4321); columns 7141(-1501); elements 30168(-5822)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     2.5399500000e+05 Pr: 1440(2.36784e+07) 0s
       1509     9.5062030527e+08 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 1509
Objective value     :  9.5062030527e+08
HiGHS run time      :          0.05


(CAPG = [15721.225806451605, 34098.174193548395], GEN = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], CURTAIL = [-0.0 -0.0 … -0.0 -0.0; 0.0 0.0 … 0.0 0.0; 17712.800000000003 17067.800000000003 … 202.59999999999854 1117.5999999999985], NSE = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], GenInvCost = 1.5e10, TIC = 1.5e10, EnergyProvisionCost = 9.308492018709673e8, UnservedEnergyCost = 0.0, CurtailedVreCost = 1.9771103400000032e7, OV = 9.506203052709668e8)

In [10]:
c1s1b05 = case1scheme1(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
budget=5e9, voll=1e6, bigm=100e3, epsilon=0.05)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
10077 rows, 7254 cols, 30282 nonzeros
10076 rows, 7141 cols, 30168 nonzeros
Presolve : Reductions: rows 10076(-4321); columns 7141(-1501); elements 30168(-5822)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     2.5399500000e+05 Pr: 1440(2.36784e+07) 0s
       2471     7.3195075713e+11 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 2471
Objective value     :  7.3195075713e+11
HiGHS run time      :          0.10


(CAPG = [25641.02564102564, 0.0], GEN = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], CURTAIL = [-0.0 -0.0 … -0.0 -0.0; 0.0 0.0 … 0.0 0.0; 17712.800000000003 17067.800000000003 … 202.59999999999854 1117.5999999999985], NSE = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], GenInvCost = 5.0e9, TIC = 5.0e9, EnergyProvisionCost = 1.181668075e9, UnservedEnergyCost = 7.307493179487184e11, CurtailedVreCost = 1.9771103400000032e7, OV = 7.319507571271179e11)

## Scheme 2 (Consider Storage)

Note: 
- 1 USD = 0.8 GBP
- 1 USD/kWh = 1000 USD/MWh
- Charging and Discharging cost: 5 USD/MWhh to promote efficient charging and discharging state

In [7]:
function case1scheme2(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
    budget, voll, bigm, epsilon)

    # SETS
    V = vreinfo.id
    G = generatorinfo.id
    S = storageinfo.id
    T = halfhourlydemand.hh
    T1 = T[2:end]

    # INITIATE MODEL
    model = Model()
    set_optimizer(model, HiGHS.Optimizer)

    # DECISION VARIABLES
    @variables(model, begin
        CAPG[G] >= 0
        CAPS[S] >= 0
        SOCM[S] >= 0
        GEN[G,T] >= 0
        SOC[S,T] >= 0
        CHARGE[S,T] >= 0
        DISCHARGE[S,T] >= 0
        CURTAIL[V,T] >= 0
        NSE[T] >= 0
    end)

    # BUDGET CONSTRAINT
    @expression(model, GenInvCost, sum(generatorinfo[generatorinfo.id.==g,:fixedCost][1] * CAPG[g] for g in G))
    @expression(model, StorPowInvCost, sum(0.8e3 * storageinfo[storageinfo.id.==s,:bestCasePowerCost][1] * CAPS[s] for s in S))
    @expression(model, StorEnInvCost, sum(0.8e3 * storageinfo[storageinfo.id.==s,:bestCaseEnergyCost][1] * SOCM[s] for s in S))
    @expression(model, TIC, GenInvCost + StorPowInvCost + StorEnInvCost)
    @constraint(model, cBudget, TIC <= budget)

    # OBJECTIVE FUNCTION
    @expression(model, EnergyProvisionCost, 
        sum(0.5 * generatorinfo[generatorinfo.id.==g,:varCost][1] * GEN[g,t] for g in G for t in T))
    @expression(model, UnservedEnergyCost, 
        sum(0.5 * voll * NSE[t] for t in T))
    @expression(model, CurtailedVreCost,
        sum(0.5 * vreinfo[vreinfo.id.==v,:varCost][1] * CURTAIL[v,t] for v in V for t in T))
    @expression(model, PenaltyChargeDischarge, 5 * sum(CHARGE[g,t] + DISCHARGE[g,t] for g in G for t in T))
    @objective(model, Min, EnergyProvisionCost + UnservedEnergyCost + CurtailedVreCost + PenaltyChargeDischarge)

    # POWER BALANCE CONSTRAINT
    @constraint(model, c01[t in T], sum(GEN[g,t] for g in G) + NSE[t] - sum(CURTAIL[v,t] for v in V) +
        sum(vreinfo[vreinfo.id.==v,:installedMW][1] * 
        halfhourlyvrecf[(halfhourlyvrecf.hh.==t) .& (halfhourlyvrecf.vre_id.==v),:cf][1] for v in V) +
        sum(DISCHARGE[s,t] for s in S) - sum(CHARGE[s,t] for s in S) == 
        halfhourlydemand[halfhourlydemand.hh.==t,:load][1])
    
    # GENERATOR LIMIT CONSTRAINT
    @constraint(model, c90[g in G, t in T], GEN[g,t] <= CAPG[g])

    # GENERATOR RAMP UP CONSTRAINT
    @constraint(model, c11[g in G, t in T1], 
        GEN[g,t] - GEN[g,t-1] <= generatorinfo[generatorinfo.id.==g,:rampupRate][1] * CAPG[g])
    
    # GENERATOR RAMP DOWN CONSTRAINT
    @constraint(model, c12[g in G, t in T1],
        GEN[g,t-1] - GEN[g,t] <= generatorinfo[generatorinfo.id.==g,:rampdownRate][1] * CAPG[g])
    
    # VRE CURTAIL CONSTRAINT
    @constraint(model, c13[v in V, t in T], CURTAIL[v,t] <= 
        vreinfo[vreinfo.id.==v,:installedMW][1] * 
        halfhourlyvrecf[(halfhourlyvrecf.hh.==t) .& (halfhourlyvrecf.vre_id.==v),:cf][1])
    
    # STORAGE CHARGE LIMIT CONSTRAINT
    @constraint(model, c92[s in S, t in T], CHARGE[s,t] <= CAPS[s])

    # STORAGE DISCHARGE LIMIT CONSTRAINT
    @constraint(model, c93[s in S, t in T], DISCHARGE[s,t] <= CAPS[s])
    
    # STORAGE SOC LIMIT CONSTRAINT
    @constraint(model, c19[s in S, t in T], SOC[s,t] <= SOCM[s])
    @constraint(model, c20[s in S], SOC[s,minimum(T)] == 0.5 * SOCM[s])
    @constraint(model, c21[s in S], SOC[s,maximum(T)] == 0.5 * SOCM[s])

    # STORAGE SOC UPDATE CONSTRAINT
    @constraint(model, c22[s in S, t in T1], SOC[s,t] == SOC[s,t-1] + 
        (CHARGE[s,t] * storageinfo[storageinfo.id.==s,:bestCaseEff][1]) - 
        (DISCHARGE[s,t] / storageinfo[storageinfo.id.==s,:bestCaseEff][1]))

    optimize!(model)

    return(
        CAPG = value.(CAPG).data,
        CAPS = value.(CAPS).data,
        SOCM = value.(SOCM).data,
        GEN = value.(GEN).data,
        SOC = value.(SOC).data,
        CHARGE = value.(CHARGE).data,
        DISCHARGE = value.(DISCHARGE).data,
        NET_DISCHARGE = value.(DISCHARGE).data .- value.(CHARGE).data,
        CURTAIL = value.(CURTAIL).data,
        NSE = value.(NSE).data,
        GenInvCost = value(GenInvCost),
        StorPowInvCost = value(StorPowInvCost),
        StorEnInvCost = value(StorEnInvCost),
        TIC = value(TIC),
        EnergyProvisionCost = value(EnergyProvisionCost),
        UnservedEnergyCost = value(UnservedEnergyCost),
        CurtailedVreCost = value(CurtailedVreCost),
        OV = objective_value(model),
    )
end

case1scheme2 (generic function with 1 method)

In [9]:
c1s2b10 = case1scheme2(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
budget=10e9, voll=1e6, bigm=100e3, epsilon=0.05)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
90675 rows, 68156 cols, 272540 nonzeros
90675 rows, 68156 cols, 272540 nonzeros
Presolve : Reductions: rows 90675(-4376); columns 68156(-994); elements 272540(-5398)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 1440(3.86983e+08) 0s
       4433     1.6796299541e+04 Pr: 33003(2.01735e+11); Du: 0(7.19476e-07) 5s
       7430     6.2599574077e+04 Pr: 20775(4.12103e+11); Du: 0(8.82986e-07) 10s
      10208     1.1774563225e+05 Pr: 19411(1.64938e+11); Du: 0(5.35038e-07) 15s
      12674     1.8910510248e+05 Pr: 27316(2.32151e+11); Du: 0(6.03302e-07) 20s
      14821     2.4517480427e+05 Pr: 26500(9.19848e+11); Du: 0(5.77963e-07) 25s
      17335     3.5377400835e+05 Pr: 24744(1.65976e+11); Du: 0(6.09254e-07) 31s
      19819     4.3015722677e+05 Pr: 22503(1.35779e+11); Du: 0(5.89193e-07) 36s
      22

(CAPG = [47979.290322580644, 1840.1096774193547], CAPS = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, -0.0, -0.0, 0.0, 0.0], SOCM = [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0], GEN = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], SOC = [-0.0 0.0 … 0.0 -0.0; -0.0 -0.0 … -0.0 -0.0; … ; -0.0 -0.0 … -0.0 -0.0; -0.0 0.0 … -0.0 -0.0], CHARGE = [0.0 0.0 … 0.0 0.0; 0.0 -0.0 … -0.0 -0.0; … ; -0.0 0.0 … -0.0 -0.0; -0.0 -0.0 … -0.0 -0.0], DISCHARGE = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 -0.0 … 0.0 0.0; 0.0 -0.0 … 0.0 0.0], NET_DISCHARGE = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 -0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], CURTAIL = [-0.0 -0.0 … -0.0 -0.0; 0.0 0.0 … 0.0 0.0; 17712.800000000003 17067.800000000003 … 202.59999999999854 1117.5999999999985], NSE = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], GenInvCost = 1.0e10, StorPowInvCost = 0.0, StorEnInvCost = 0.0, TIC = 1.0e10, EnergyProvi

In [10]:
c1s2b20 = case1scheme2(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
budget=20e9, voll=1e6, bigm=100e3, epsilon=0.05)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
90675 rows, 68156 cols, 272540 nonzeros
90675 rows, 68156 cols, 272540 nonzeros
Presolve : Reductions: rows 90675(-4376); columns 68156(-994); elements 272540(-5398)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 1440(3.86983e+08) 0s
       4785     2.3268108137e+04 Pr: 16589(2.49315e+11); Du: 0(6.41748e-07) 5s
       7866     6.2276288923e+04 Pr: 19862(1.3167e+11); Du: 0(7.64953e-07) 10s
      10712     1.3660476839e+05 Pr: 25051(5.15778e+11); Du: 0(3.54787e-07) 15s
      12940     1.8492265673e+05 Pr: 21120(8.68397e+11); Du: 0(7.40301e-07) 21s
      15289     2.2723526858e+05 Pr: 22721(9.61969e+11); Du: 0(9.16302e-07) 26s
      17546     2.6408114791e+05 Pr: 27421(4.27874e+11); Du: 0(5.42089e-07) 32s
      19595     2.8779309597e+05 Pr: 33419(9.57883e+11); Du: 0(9.7809e-07) 37s
      2173

(CAPG = [4357.88057476665, 24147.563702701395], CAPS = [-0.0, 12915.264351236765, -0.0, -0.0, 2738.291371295227, 0.0, 0.0, 0.0, -0.0, -0.0, 5660.399999999995, -0.0, -0.0, -0.0], SOCM = [-0.0, 2.268530961275762e6, -0.0, -0.0, 199584.2600355829, -0.0, -0.0, -0.0, -0.0, -0.0, 66998.18181818399, 0.0, -0.0, -0.0], GEN = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], SOC = [-0.0 -0.0 … 0.0 -0.0; 1.134265480637881e6 1.1429187077532096e6 … 1.1335166886378808e6 1.134265480637881e6; … ; -0.0 -0.0 … -0.0 -0.0; -0.0 -0.0 … -0.0 -0.0], CHARGE = [0.0 0.0 … -0.0 -0.0; 0.0 12915.264351236765 … 202.59999999978035 1117.6000000001973; … ; -0.0 -0.0 … 0.0 0.0; -0.0 -0.0 … 0.0 0.0], DISCHARGE = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 -0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], NET_DISCHARGE = [0.0 0.0 … 0.0 0.0; 0.0 -12915.264351236765 … -202.59999999978035 -1117.6000000001973; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], CURTAIL = [-0.0 -0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 9314.10862870478 1414.2442774680112 … 0.0 0.

In [11]:
c1s2b30 = case1scheme2(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
budget=30e9, voll=1e6, bigm=100e3, epsilon=0.05)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
90675 rows, 68156 cols, 272540 nonzeros
90675 rows, 68156 cols, 272540 nonzeros
Presolve : Reductions: rows 90675(-4376); columns 68156(-994); elements 272540(-5398)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 1440(3.86983e+08) 0s
       5098     2.2840989496e+04 Pr: 19448(7.14747e+11); Du: 0(7.08003e-07) 5s
       8180     5.8116964993e+04 Pr: 18711(1.07603e+12); Du: 0(1.07655e-06) 10s
      10933     1.0042931003e+05 Pr: 24697(3.28748e+11); Du: 0(1.84694e-06) 16s
      13435     1.5420963232e+05 Pr: 29198(6.7599e+11); Du: 0(1.5988e-06) 21s
      15959     2.0207425955e+05 Pr: 19400(2.55388e+12); Du: 0(1.78856e-06) 27s
      18219     2.3840208209e+05 Pr: 20057(2.9745e+11); Du: 0(1.54661e-06) 32s
      20310     2.5856761546e+05 Pr: 23966(6.37266e+11); Du: 0(1.33068e-06) 37s
      22280

(CAPG = [0.0, 22613.583330876034], CAPS = [-0.0, 17411.075002831512, 0.0, 0.0, -0.0, 0.0, -0.0, 0.0, 11729.00184342062, 1.990533994642598e-10, -0.0, -0.0, -0.0, -0.0], SOCM = [-0.0, 3.0817847774115177e6, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 821362.6591920853, 2.22939807399971e-9, -0.0, -0.0, -0.0, -0.0], GEN = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], SOC = [-0.0 -0.0 … -0.0 -0.0; 1.5408923887057588e6 1.552557808957656e6 … 1.5408923887057588e6 1.5408923887057588e6; … ; -0.0 -0.0 … -0.0 -0.0; -0.0 -0.0 … -0.0 -0.0], CHARGE = [0.0 0.0 … -0.0 -0.0; 0.0 17411.075002831512 … 0.0 0.0; … ; -0.0 0.0 … 0.0 0.0; -0.0 0.0 … 0.0 0.0], DISCHARGE = [0.0 -0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], NET_DISCHARGE = [0.0 -0.0 … 0.0 0.0; 0.0 -17411.075002831512 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], CURTAIL = [-0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 5983.798156579183 0.0 … 0.0 0.0], NSE = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0

In [8]:
c1s2b15 = case1scheme2(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
budget=15e9, voll=1e6, bigm=100e3, epsilon=0.05)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
90675 rows, 68156 cols, 272540 nonzeros
90675 rows, 68156 cols, 272540 nonzeros
Presolve : Reductions: rows 90675(-4376); columns 68156(-994); elements 272540(-5398)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 1440(3.86983e+08) 0s
       4634     1.7675983110e+04 Pr: 15319(2.29401e+11); Du: 0(8.71912e-07) 5s
       7649     4.4472796588e+04 Pr: 15662(4.31636e+11); Du: 0(9.84596e-07) 10s
      10631     9.0643870048e+04 Pr: 20067(2.7671e+12); Du: 0(1.14006e-06) 16s
      12813     1.6657033758e+05 Pr: 26287(1.36496e+12); Du: 0(1.08266e-06) 21s
      15021     2.2090902321e+05 Pr: 25925(7.35518e+11); Du: 0(1.26111e-06) 27s
      17288     2.6304628975e+05 Pr: 24793(7.35113e+11); Du: 0(1.20285e-06) 32s
      19565     3.0241452151e+05 Pr: 24217(1.84696e+11); Du: 0(8.88165e-07) 37s
      217

(CAPG = [16645.08900346422, 20731.0], CAPS = [-0.0, 513.5246940381862, 0.0, 0.0, 5921.275305961817, 0.0, -0.0, -0.0, -0.0, -0.0, 6008.510996535779, -0.0, -0.0, -0.0], SOCM = [-0.0, 105070.00606286977, -0.0, -0.0, 488271.92461512337, -0.0, -0.0, -0.0, -0.0, -0.0, 76750.17806844346, -0.0, -0.0, -0.0], GEN = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], SOC = [-0.0 -0.0 … 0.0 -0.0; 52535.003031434884 52879.06457644047 … 52190.9414864293 52535.003031434884; … ; -0.0 -0.0 … -0.0 -0.0; -0.0 -0.0 … -0.0 -0.0], CHARGE = [0.0 -0.0 … -0.0 -0.0; 0.0 513.5246940381862 … 202.59999999999854 513.5246940381862; … ; -0.0 -0.0 … 0.0 0.0; -0.0 -0.0 … 0.0 0.0], DISCHARGE = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 -0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], NET_DISCHARGE = [0.0 0.0 … 0.0 0.0; 0.0 -513.5246940381862 … -202.59999999999854 -513.5246940381862; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], CURTAIL = [-0.0 -0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 5783.013697502407 10632.999999999998 … 0.0 0.0], NSE = [0.0, 0.0, 

In [9]:
c1s2b05 = case1scheme2(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
budget=5e9, voll=1e6, bigm=100e3, epsilon=0.05)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
90675 rows, 68156 cols, 272540 nonzeros
90675 rows, 68156 cols, 272540 nonzeros
Presolve : Reductions: rows 90675(-4376); columns 68156(-994); elements 272540(-5398)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 1440(3.86983e+08) 0s
       4408     2.3294159523e+04 Pr: 21409(5.98784e+10); Du: 0(5.0935e-07) 5s
       7537     8.9571933909e+04 Pr: 27840(3.96836e+11); Du: 0(7.73119e-07) 10s
      10247     1.4356765499e+05 Pr: 27189(1.68658e+11); Du: 0(5.67358e-07) 15s
      12915     2.6172451244e+05 Pr: 25916(8.26153e+11); Du: 0(7.2771e-07) 21s
      15403     3.8388050148e+05 Pr: 27538(2.73307e+11); Du: 0(8.78975e-07) 26s
      17568     4.7801398024e+05 Pr: 24328(3.15065e+11); Du: 0(8.42818e-07) 32s
      20039     5.6345022390e+05 Pr: 22150(2.19625e+11); Du: 0(9.48639e-07) 38s
      2213

(CAPG = [25641.02564102564, 0.0], CAPS = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, -0.0, 0.0, 0.0, 0.0], SOCM = [-0.0, -0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, 0.0], GEN = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … -0.0 -0.0], SOC = [-0.0 -0.0 … -0.0 -0.0; -0.0 -0.0 … -0.0 -0.0; … ; -0.0 -0.0 … -0.0 -0.0; -0.0 -0.0 … -0.0 -0.0], CHARGE = [0.0 0.0 … 0.0 0.0; 0.0 -0.0 … -0.0 -0.0; … ; -0.0 -0.0 … -0.0 -0.0; -0.0 -0.0 … -0.0 -0.0], DISCHARGE = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … -0.0 0.0; 0.0 -0.0 … -0.0 -0.0], NET_DISCHARGE = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], CURTAIL = [-0.0 -0.0 … -0.0 -0.0; 0.0 0.0 … 0.0 0.0; 17712.800000000003 17067.800000000003 … 202.59999999999854 1117.5999999999985], NSE = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], GenInvCost = 5.0e9, StorPowInvCost = 0.0, StorEnInvCost = 0.0, TIC = 5.0e9, EnergyProvisionCost = 1.1